In [1]:
from research.research_engine import build_ResearchEngine
from strategies.vault_rebalancing import YieldStrategy
from strategies.vault_backtest import VaultBacktestEngine
import json
import pandas as pd
from copy import deepcopy

In [2]:
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [11]:
parameters = {
    "input_data": {
        "ResearchName": "defillama",
        "dirpath": ["signal","scrappers","defillama_history","data","DynLst"],
        "start_date": "2022-02-15T00:00:00",
        "selected_instruments": [
            "747c1d2a-c668-4682-b9f9-296708a3dd90",
            "d4b3c522-6127-4b89-bedf-83641cdcd2eb",
            "e6435aae-cbe9-4d26-ab2c-a4d533db9972",
            "77020688-e1f9-443c-9388-e51ace15cc32",
            "e880e828-ca59-4ec6-8d4f-27182a4dc23d",
            "6d342d6a-9796-443e-8152-b94e8b6021fc",
            "5ce23e7e-3800-4c9c-ad30-6db3db0515a1",
            "42c02674-f7a2-4cb0-be3d-ade268838770",
            "e378a7c2-6285-4993-9397-87ac9c8adc15",
            "f1a22245-a1f1-4389-8ff2-dde4914b272d",
            "4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b",
            "fc7701fc-e290-4606-a8be-9e4ba6c5f91a",
            "c9873dab-0979-478c-b48c-3c3a0c935449",
            "e72916f7-a2d1-47ad-a4b9-2b054337cfd6" # instadapp
        ]
    },
    "run_parameters": {
        "unit_test": False,
        "models": {
            "haircut_apy": {
                "TrivialEwmPredictor": {
                    "params": {"halflife": "7d", "cap": 0.3}
                }
            }
        }
    },
    "feature_map": {
        "haircut_apy": {
            "ewma_expansion": {
                "windows":[2]},
        }
    },
    "label_map": {
        "haircut_apy": {
            "horizons": [7]
        }
    },
    "strategy": {
        "initial_wealth": 100,
        "cost": [0.001],
        "verbose": "interim",
        "warm_start": False,
        "finite_diff_rel_step": 1e-2,
        "ftol": 1e-2
    },
    "backtest": {
        "start_date": "2023-02-15T00:00:00",
        "end_date": "2023-08-15T00:00:00"
    }
}

In [12]:
metadata = pd.read_csv("C:/Users/david/signal/scrappers/defillama_history/data/DynLst_pool_metadata.csv")
pool_map = metadata.loc[metadata['pool'].isin(parameters['input_data']['selected_instruments']),['pool','project','symbol']].set_index('pool').T.to_dict()
pool_map = {key: '{}_{}'.format(x['project'], x['symbol']) for key, x in pool_map.items()}
index_map = {f'weight_{i}': f'w_{value}' for i, value in enumerate(pool_map.values())}
index_map |= {f'yield_{i}': f'y_{value}' for i, value in enumerate(pool_map.values())} 
pool_map

{'747c1d2a-c668-4682-b9f9-296708a3dd90': 'lido_STETH',
 'd4b3c522-6127-4b89-bedf-83641cdcd2eb': 'rocket-pool_RETH',
 'e6435aae-cbe9-4d26-ab2c-a4d533db9972': 'aave-v3_WSTETH',
 '77020688-e1f9-443c-9388-e51ace15cc32': 'frax-ether_SFRXETH',
 'e880e828-ca59-4ec6-8d4f-27182a4dc23d': 'aave-v3_WETH',
 '6d342d6a-9796-443e-8152-b94e8b6021fc': 'convex-finance_ETH-FRXETH',
 '5ce23e7e-3800-4c9c-ad30-6db3db0515a1': 'convex-finance_ETH-STETH',
 '42c02674-f7a2-4cb0-be3d-ade268838770': 'balancer-v2_RETH-WETH',
 'e378a7c2-6285-4993-9397-87ac9c8adc15': 'convex-finance_ETH-STETH',
 'f1a22245-a1f1-4389-8ff2-dde4914b272d': 'aave-v3_RETH',
 '4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b': 'balancer-v2_WSTETH-WETH',
 'fc7701fc-e290-4606-a8be-9e4ba6c5f91a': 'convex-finance_ETH-RETH',
 'c9873dab-0979-478c-b48c-3c3a0c935449': 'convex-finance_STETH-FRXETH'}

In [5]:
engine = build_ResearchEngine(parameters)
data = pd.DataFrame(engine.performance)
data = data.ffill().dropna()

fit engine


In [6]:
parameter_grid = {"cap": [100],
                  "haflife": ["7d","10d", "14d","21d"],
                  "cost": [0.001],
                  "assumed_holding_days": [5,7,10,14,21,30]}
parameter_grid = {"cap": [100],
                  "haflife": ["7d"],
                  "cost": [0.001],
                  "assumed_holding_days": [90]}

# create parameters_list as a list of dicts from parameter_grid
original_parameter = parameters
parameter_dict = dict()
for cap in parameter_grid["cap"]:
    for haflife in parameter_grid["haflife"]:
        for cost in parameter_grid["cost"]:
            for assumed_holding_days in parameter_grid["assumed_holding_days"]:
                new_parameter = deepcopy(original_parameter)
                new_parameter["run_parameters"]["models"]["haircut_apy"]["TrivialEwmPredictor"]["params"]['cap'] = cap
                new_parameter["run_parameters"]["models"]["haircut_apy"]["TrivialEwmPredictor"]["params"]['halflife'] = haflife
                new_parameter['strategy']['cost'] = cost
                new_parameter["label_map"]["haircut_apy"]["horizons"] = [assumed_holding_days]

                name = (cap, haflife, cost, assumed_holding_days)

                parameter_dict[name]= new_parameter

backtest = VaultBacktestEngine(parameters['backtest'])
result = dict()
for name, cur_params in parameter_dict.items():
    engine = build_ResearchEngine(cur_params)
    vault_rebalancing = YieldStrategy(cur_params['strategy'],
                                      features=data,
                                      research_engine=engine)
    result[name] = backtest.run(vault_rebalancing)

ValueError: Found array with 0 sample(s) (shape=(0, 13)) while a minimum of 1 is required by StandardScaler.

In [ ]:
result[(100, '7d', 0.001, 30)]['weights'].T.rename(columns=index_map).iplot()

In [ ]:
result[(100, '7d', 0.001, 30)]['yields'].T.rename(columns=index_map).iplot()